# Introduction to Language Modeling in Python

This notebook will cover the basics of using near state-of-the-art models in natural language processing. The library allows users to submit open source models  (on [huggingface](https://huggingface.co/)) with trained weights for others to reuse. Models on this site are of the transformer architecture - hence the library is called "transformers".

Language models are used as the foundation for further tasks in NLP as they capture the concept of language which is valuable for use in tasks such as named entity recognition (identifying people, places, or organisations), sentiment analysis (determing whether text is positive or negative) or other text classification tasks.

In [ ]:
# Sets up python environment and installs dependencies
!python3 -m venv lms_venv
!source lms_venv/bin/activate
!wget https://raw.githubusercontent.com/wrmthorne/LMs_Session/main/requirements.txt
%pip install -r requirements.txt

In [ ]:
# Import packages
from transformers import AutoModel, AutoTokenizer, pipeline, utils, T5ForConditionalGeneration
from bertviz import model_view, head_view
from bertviz.neuron_view import show
from bertviz.transformers_neuron_view import BertModel, BertTokenizer

utils.logging.set_verbosity_error()

## Tokenising

Computer models cannot understand or perform mathematical operations on words so they need to be represented in a machine readable format. Tradiationally, this was done by giving each unique word a number (index). This has a problem. To represent all words that have and will ever exist, you need infinitely many indices which is impossible to compute.

Instead, one method called word-piece was developed which builds a vocabulary with sub-word "tokens" which are then converted to indices. The algorithm starts with the letters of the alphabet, numbers and puctuation and finds how to combine characters into larger chunks based on how many times they appear together in text. This limits the vocabulary size to an upper limit while being able to produce every word (and misspellings) in the language.

e.g. the word "WANDISCO" may be split into "WA ##ND ##IS ##CO" (## indicates that this is stuck to the end of another token). We aren't likely to see the full word as many times as other words so it won't be in the vocabulary but the subcomponents may appead in other words too so they are more probably.

The transformers library provides us with pretrained tokenisers for us to use for their matching model. If you wanted to train your own model, you can also train a tokeniser to go with it. For now we will use the BERT model as an example.

BERT also adds two extra tokens to your input string. `[CLS]` which is used for tasks that build on top of the language model such as predicting missing words and `[SEP]` which is used to indicate the end of a sentence.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# Feel free to change the test string
test_string = 'WANDISCO'

token_ids = tokenizer.encode(test_string)
print(f'IDS: {token_ids}')

tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(f'Tokens: {tokens}')

## Model

The language model itself doesn't do much. It is effectively a big memory unit for how language functions, ontop of which "heads" are applied. A head is a new network which is design specifically to solve a task. One such task - which is how the model is trained - is to predict a missing word in a sequence. This is called `masked language modelling`. We cna find out what type of company BERT thinks WANDISCO is by using the `[MASK]` token on the word we want it to predict.

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-cased')

test_string = 'WANDISCO is a [MASK] company'
unmasker(test_string)

## Bias

The model is only as good as the data it is trained on and the model likely has inherent bias. Most models are trained on internet data or books. Therefore, the views of the model will reflect that of the internet. E.g. comparing what the model predicts as an occupation for men and women:

In [ ]:
male_preds = unmasker('The man worked as a [MASK].')
female_preds = unmasker('The woman worked as a [MASK].')

print(f'Male predictions: {[pred["token_str"] for pred in male_preds]}')
print(f'Female predictions: {[pred["token_str"] for pred in female_preds]}')

## Attention

The transformer architecture makes use of "attention" which is a way of learning which words in a sentence a different word is focusing on. e.g.

"The rabbit ran up the hill and the fox followed"

The word rabbit would likely pay more attention to the words "The" as it is the article for the word "rabbit" and "ran" is the verb associated with the rabbit. It is less likely to pay attention to the "the" preceding "fox" as it is the article for the fox rather than the rabbit.

Transformer models like BERT make use of many attention blocks (layers) and within each block, use many attention heads. Each of these blocks and heads should ideally look at a different concept.

Inspecting Layer 2 Head 3 in BERT, we can see that it has learned that each word should look to the next word in the sequence. Layer 6 Head 4 learns that words are part of the same sentence and the assignment or articles to their nouns (e.g. 'the' -> 'rabbit'). Have a look through the layers and heads and try and find any obvious properties learned in a particular head (many of them will appear random).

In [ ]:
model_type = 'bert'
model_version = 'bert-base-cased'
sentence_a = "The rabbit ran up the hill and the fox followed"

bert_model = BertModel.from_pretrained(model_version, output_attentions=True)
bert_tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=False)
show(bert_model, model_type, bert_tokenizer, sentence_a, layer=2, head=0)

We can get an overview of what every head is doing in every layer by plotting them all. Many of them link the whole sequence to the `[SEP]` token and many other link to the `[CLS]` token. These tokens are called "special tokens" as they are used for extra functions beyond modelling language. As mentioned before, `[CLS]` is for classification and use in guiding the heads which extend the model and `[SEP]` is important for defining sentence boundaries.

In [ ]:
model = AutoModel.from_pretrained('bert-base-cased', output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


inputs = tokenizer.encode(sentence_a, return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1] 
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

model_view(attention, tokens)

Before running the next cell, think about what would happen if we put two sentences in the sequence. We would have two `[SEP]` tokens because we have two sentences but only 1 `[CLS]` token because they are part of the same sequence. What do you think the attention diagrams will look like?

Run the cell and find out if it is what you expected? Try to think about why?

In [ ]:
sentence_b = "The rabbit ran up the hill and the fox did not follow"

inputs = tokenizer.encode(sentence_a, sentence_b, return_tensors='pt')

outputs = model(inputs)
attention = outputs[-1] 
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
print(' '.join(tokens))

model_view(attention, tokens)

## Different Architectures

There are 3 main types of transformer architecture: encoder-decoder, encoder-only and decoder-only. [BERT](https://huggingface.co/bert-base-cased) is an encoder only model. It encodes the text into a machine understandable format called a latent space. This latent space is accessible in BERT through the CLS token which allows another neural network to use its language modelling to perform more tasks.

[GPT](https://huggingface.co/gpt2) is a decoder only model. It decodes the input text directly into the output text.

BARD and [T5](https://huggingface.co/t5-base) are encoder-decoder models. The encoder translates the text into the latent space and this latent space is read by the decoder. This makes these models especially useful for translation.

To generate text, a decoder is needed, hence why BERT cannot be used for text generation.

Let's have a look at T5 for text generation. The authors of T5 claimed that any task can be reformulated as a text to text task. T5 uses a system called prefix and prompt. The model is trained to perform many tasks including translation, question & answering, and classification tasks. The task you want it to perform is defined by a prefix e.g. "Translate English to German: " and the content you want the task performed on is the prompt e.g. "I flew to Germany"

In [ ]:
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small', output_attentions=True)
t5_tokenizer = AutoTokenizer.from_pretrained('t5-small', model_max_length=512)

input = t5_tokenizer('translate English to German: I flew to Germany.', return_tensors='pt')
output_ids = t5_model.generate(**input, max_length=512) # ** is used to unpack the dictionary

output_tokens = t5_tokenizer.batch_decode(output_ids)
output_tokens

Have a go at some other prefixes and prompts to see what T5 can do.

Some of the prefixes are:
- translate English to German
- summarize
- answer question
- predict sentiment
- binary classification
- generate question

In [ ]:
prompt = 'summarize:'
prefix = 'This is a long story about the rabbit and the fox. The rabbit ran up the hill to escape the fox. One time the fox followed. The other time, it did not.'

input = t5_tokenizer(prompt + ' ' + prefix, return_tensors='pt')
output_ids = t5_model.generate(**input, max_length=512)

output_tokens = t5_tokenizer.batch_decode(output_ids)
output_tokens

We can also inspect the attention diagrams for T5. Because T5 is an encoder-decoder model, we have the encoder attentions, the decoder attentions and a series of cross attentions which map between the attention blocks.

In [ ]:
encoder_input_ids = t5_tokenizer("translate English to German: I flew to Germany.", return_tensors="pt", add_special_tokens=True).input_ids
decoder_input_ids = t5_tokenizer(text_target="Ich flog nach Deutschland.", return_tensors="pt", add_special_tokens=True).input_ids

outputs = t5_model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = t5_tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = t5_tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens=encoder_text,
    decoder_tokens=decoder_text
)